# F1 EDA
#### PART 1: 

Here I answer some questions regarding drivers, races, and seasons.

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://i.giphy.com/media/v1.Y2lkPTc5MGI3NjExNWl6ajhnaDU4aHVteGpmamk2Z3BkdXd3NDFwY3BybG9uMHB2MzZtcSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/sA50TO4Ew7QXu/giphy.gif")

## 1) Importing Packages and Data

In [2]:
import pandas as pd
import numpy as np
import F1_Preprocessing #Internal package to preprocess data
np.random.seed(1994)

In [3]:
#The data stored in this path is obtained from the API of https://ergast.com/mrd/. It is continuously updated.
#To update this data please run the file "1_API_Requests_Ergast.py"

path = 'C:/Users/gabri/Dropbox/Gaby/Proyectos/My_Portafolio/F1/Data/'

ResultsDF=pd.read_csv(path+"ResultsDF.csv")
QualifyingDF=pd.read_csv(path+"QualifyingDF.csv")
LapsDF=pd.read_csv(path+"LapsDF.csv")
PitsDF=pd.read_csv(path+"PitsDF.csv")

C:\Users\gabri\AppData\Local\Temp\ipykernel_17052\478565046.py:8: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  LapsDF=pd.read_csv(path+"LapsDF.csv")


## 2) Data Preprocessing

In [4]:
# Apply the data preprocessing for each df. 
# This is done this way so the preprocessing is consistent thorughout all notebooks.
ResultsDF=F1_Preprocessing.preprocess_F1results(ResultsDF)
LapsDF=F1_Preprocessing.preprocess_F1laps(LapsDF)
PitsDF=F1_Preprocessing.preprocess_F1pits(PitsDF)

# Merging features between dfs
Laps_Pits=pd.merge(LapsDF,PitsDF[["season","round","driverid","lap_number","pit_stop_time","pit_stop_duration_in_milliseconds","pit_stop_number"]],
                   left_on=["driverid","season","round","current_lap_number"],
                   right_on=["driverid","season","round","lap_number"],how="left")

# We only have information of the pit stops from 2012, so Laps_Pits will have instances from 2012 or later
Laps_Pits=Laps_Pits[Laps_Pits["season"]>=2012]